In [2]:
import sys


def validate_float(prompt, min_value=0, max_value=float('inf')):
    while True:
        try:
            value = float(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print(f"Please enter a value between {min_value} and {max_value}.")
        except ValueError:
            print("Please enter a valid number.")

def validate_choice(prompt, choices):
    choice_map = {str(i+1): c for i, c in enumerate(choices)}
    while True:
        print(prompt)
        for k, v in choice_map.items():
            print(f"{k}. {v}")
        selection = input("Enter choice number: ")
        if selection in choice_map:
            return choice_map[selection]
        else:
            print("Invalid choice. Please try again.")

def choose_unit_system():
    choice = input("Choose your unit system: [1] Imperial (miles, gallons), [2] Metric (km, liters): ").strip()
    return 'metric' if choice == '2' else 'imperial'

def transportation(unit_system):
    print("\n(Transportation)")
    if unit_system == 'imperial':
        miles = validate_float("How many miles do you drive in a typical week? ")
        mpg = validate_float("What’s your car’s fuel efficiency in miles per gallon (mpg)? ", 5, 100)
        annual_miles = miles * 52
        emissions = (annual_miles / mpg) * 8.89 / 1000
        return emissions, f"{annual_miles:.0f} miles/year"
    else:
        km = validate_float("How many kilometers do you drive per week? ")
        l_per_100km = validate_float("What’s your car’s fuel efficiency in liters per 100 kilometers? ", 2, 30)
        liters_per_km = l_per_100km / 100
        annual_km = km * 52
        emissions = annual_km * liters_per_km * 2.31 / 1000
        return emissions, f"{annual_km:.0f} km/year"

def energy():
    print("\n(Home Energy)")
    monthly_kwh = validate_float("About how many kilowatt-hours (kWh) of electricity do you use each month? ", 0)
    annual_kwh = monthly_kwh * 12
    elec_emission_factor = 0.92e-3
    return annual_kwh * elec_emission_factor, f"{annual_kwh:.0f} kWh/year"

def diet():
    print("\n(Diet)")
    diet_type = validate_choice("Which option best describes your usual diet:", ["Vegan", "Vegetarian", "Pescatarian", "Omnivore"])
    emission_factors = {
        "Vegan": 1.5,
        "Vegetarian": 1.7,
        "Pescatarian": 1.9,
        "Omnivore": 2.5
    }
    return emission_factors[diet_type], diet_type

def waste():
    print("\n(Waste)")
    recycling = validate_choice("Do you recycle regularly?", ["Yes", "No"])
    waste_emission = 0.25 if recycling == "Yes" else 0.6
    return waste_emission, recycling

def summarize(emissions, highlight_threshold=0.25):
    total = sum(val[0] for val in emissions.values())
    print("\n===== Annual Carbon Footprint Summary =====")
    for category, (value, detail) in emissions.items():
        percent = value / total
        marker = "!!!" if percent >= highlight_threshold else "   "
        print(f"{marker} {category}: {value:.2f} metric tons CO2e ({detail})")
    print(f"\nTOTAL: {total:.2f} metric tons CO2e")
    print(f"Estimated impact: driving {total * 2500:.0f} miles or powering {total / 4.6:.1f} homes/year.")
    print(f"Comparison to U.S. Average (16.5 tons): {total / 16.5 * 100:.1f}%")
    earths = total / 1.7
    print(f"If everyone lived like you, we’d need {earths:.1f} Earths.")
    return total

def export_summary(emissions, total):
    filename = input("Enter a filename to save your report (e.g., my_footprint.txt): ").strip()
    if not filename:
        filename = "carbon_footprint_report.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("Carbon Footprint Report\n")
        f.write("=======================\n")
        for category, (value, detail) in emissions.items():
            f.write(f"{category}: {value:.2f} metric tons CO2e ({detail})\n")
        f.write(f"\nTOTAL: {total:.2f} metric tons CO2e\n")
        f.write("U.S. Average: 16.5 tCO2e\n")
        f.write(f"You are at {total / 16.5 * 100:.1f}% of the U.S. average.\n")
    print(f"Report saved to {filename}")

def recommend(emissions, threshold=0.25):
    total = sum(val[0] for val in emissions.values())
    print(f"\nDetailed Emission Breakdown (threshold: {threshold*100:.0f}%)")
    suggestions = {
        "Transportation": "Consider carpooling, using public transport, or switching to an EV.",
        "Energy": "Switch to LED bulbs, unplug devices, or explore renewable energy.",
        "Diet": "Try reducing meat intake or adding plant-based meals.",
        "Waste": "Recycle regularly and reduce single-use plastics."
    }
    flagged = False
    for category, (value, _) in emissions.items():
        percent = value / total
        if percent >= threshold:
            print(f"- {category}: {value:.2f} tons ({percent*100:.1f}%) — {suggestions[category]}")
            flagged = True
    if not flagged:
        print("All categories are below the threshold. Keep up the good habits!")

def show_rank(total):
    print("\n===== Performance Ranking =====")
    avg_us = 16.5
    delta = avg_us - total
    if total >= avg_us:
        print("You are above the U.S. average. Try the suggestions to reduce your impact.")
    elif total > 12:
        print("Bronze Rank — You're doing better than many Americans.")
    elif total > 8:
        print("Silver Rank — Great work reducing your footprint!")
    else:
        print("Gold Rank — You're ahead of the curve! Keep leading by example.")
    print(f"You're emitting {delta:.2f} tons less than the average American.")

def apply_suggestions(emissions):
    print("\nApplying smart suggestions to simulate reduced emissions...")
    if emissions["Transportation"][0] > 0:
        emissions["Transportation"] = (emissions["Transportation"][0] * 0.7, emissions["Transportation"][1])
    if emissions["Energy"][0] > 0:
        emissions["Energy"] = (emissions["Energy"][0] * 0.8, emissions["Energy"][1])
    if emissions["Diet"][1] != "Vegan":
        emissions["Diet"] = (1.5, "Improved (Vegan)")
    if emissions["Waste"][1] == "No":
        emissions["Waste"] = (0.25, "Now Recycling")
    print("Reductions applied. Here's your updated footprint:")

def simulate_improvements(emissions):
    print("\nWould you like to simulate the suggested improvements to see how much better you'd do?")
    choice = input("Proceed with simulation? (y/n): ").strip().lower()
    if choice != 'y':
        return
    apply_suggestions(emissions)
    new_total = summarize(emissions)
    show_rank(new_total)
    if input("Would you like to save your improved results? (y/n): ").strip().lower() == 'y':
        export_summary(emissions, new_total)

def main():
    print("Welcome to the Carbon Footprint Calculator. This tool helps you estimate your annual carbon emissions and see how your lifestyle compares to the average. Let's get started.")
    unit_system = choose_unit_system()
    emissions = {
        "Transportation": transportation(unit_system),
        "Energy": energy(),
        "Diet": diet(),
        "Waste": waste()
    }
    total = summarize(emissions)
    recommend(emissions)
    show_rank(total)
    if input("Would you like to save your results as a text file? (y/n): ").strip().lower() == 'y':
        export_summary(emissions, total)
    if input("Want to experiment with changes on your own before seeing suggestions? (y/n): ").strip().lower() == 'y':
        pass  # Placeholder for manual testing feature
    simulate_improvements(emissions)
    if input("Would you like a 30-day carbon reduction challenge with simple, daily actions you can try? (y/n): ").strip().lower() == 'y':
        pass  # Placeholder for carbon challenge feature
    print("\nThanks for using the Carbon Footprint Calculator. Every small change adds up. Good luck on your sustainability journey!")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nExited by user.")
        sys.exit(0)


Welcome to the Carbon Footprint Calculator. This tool helps you estimate your annual carbon emissions and see how your lifestyle compares to the average. Let's get started.



(Transportation)

(Home Energy)

(Diet)
Which option best describes your usual diet:
1. Vegan
2. Vegetarian
3. Pescatarian
4. Omnivore

(Waste)
Do you recycle regularly?
1. Yes
2. No

===== Annual Carbon Footprint Summary =====
!!! Transportation: 3.08 metric tons CO2e (10400 miles/year)
!!! Energy: 5.52 metric tons CO2e (6000 kWh/year)
    Diet: 2.50 metric tons CO2e (Omnivore)
    Waste: 0.25 metric tons CO2e (Yes)

TOTAL: 11.35 metric tons CO2e
Estimated impact: driving 28380 miles or powering 2.5 homes/year.
Comparison to U.S. Average (16.5 tons): 68.8%
If everyone lived like you, we’d need 6.7 Earths.

Detailed Emission Breakdown (threshold: 25%)
- Transportation: 3.08 tons (27.1%) — Consider carpooling, using public transport, or switching to an EV.
- Energy: 5.52 tons (48.6%) — Switch to LED bulbs, unplug devices, or explore renewable energy.

===== Performance Ranking =====
Silver Rank — Great work reducing your footprint!
You're emitting 5.15 tons less than the average Americ